# Prototype_2
### 추출해와서 출력하기

#### Requirements

In [1]:
import os
import glob

import numpy as np
import pandas as pd
import json

import mediapipe as mp
import cv2
import pytube

#### Prototype

In [7]:
class JustDDance:
    def __init__(self):
        self.__download_path = "./video/"
        self.__keyporint_extraction_path = "./keypoint_extraction/"
        self.__dance_shape = []
    
    def download_video(self):
        url = input("연습할 춤의 유튜브 링크: ")
        yt = pytube.YouTube(url)
        stream = yt.streams.filter(res="720p").first()
        stream.download(self.__download_path)
    
    def extract_keypoint(self, video_path):
        mp_drawing = mp.solutions.drawing_utils
        mp_drawing_style = mp.solutions.drawing_styles
        mp_holistic = mp.solutions.holistic
        
        keypoint_dict_pose = []
        keypoint_dict_left_hand = []
        keypoint_dict_right_hand = []
        keypoint_dict = {}
        
        try: cap = cv2.VideoCapture(video_path)
        except: return -1
        
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while cap.isOpened():
                success, image = cap.read()
                if not success: break
                self.__dance_shape = image.shape
                image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = holistic.process(image)
                
                try:
                    annotated_pose_landmarks = {str(idx): [lmk.x, lmk.y, lmk.z] for idx, lmk in enumerate(results.pose_landmarks.landmark)}
                    keypoint_dict_pose.append(annotated_pose_landmarks)
                    annotated_left_hand_landmarks = {str(idx): [lmk.x, lmk.y, lmk.z] for idx, lmk in enumerate(results.left_hand_landmarks.landmark)}
                    keypoint_dict_left_hand.append(annotated_left_hand_landmarks)
                    annotated_right_hand_landmarks = {str(idx): [lmk.x, lmk.y, lmk.z] for idx, lmk in enumerate(results.right_hand_landmarks.landmark)}
                    keypoint_dict_right_hand.append(annotated_right_hand_landmarks)
                except:
                    pass
                keypoint_dict = {"pose": keypoint_dict_pose, "left_hand": keypoint_dict_left_hand, "right_hand": keypoint_dict_right_hand}
                
                # image.flags.writeable = True
                # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                # mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                #                         landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 178), thickness=1, circle_radius=1),
                #                         connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 178), thickness=2, circle_radius=1))
                # mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                #                         landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 178), thickness=1, circle_radius=1),
                #                         connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 178), thickness=2, circle_radius=1))
                # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                #                         landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 102), thickness=1, circle_radius=1),
                #                         connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 102), thickness=3, circle_radius=1))
                
                # cv2.imshow("KeypointExtraction", image)
                
                # if cv2.waitKey(int(cap.get(cv2.CAP_PROP_FPS)))&0xFF==ord("q"): break
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        with open(self.__keyporint_extraction_path+video_path.split("/")[2].split(".")[0]+"_keypoints.json", "w") as fp:
            json.dump(keypoint_dict, fp)
            
        


In [8]:
jd = JustDDance()

jd.extract_keypoint("./video/[주간아 직캠] IVE YUJIN - LOVE DIVE (아이브 유진 - 러브 다이브) l EP556.mp4")